In [1]:
%pip install wikipedia

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
import os
import requests

import wikipedia 

from bs4 import BeautifulSoup

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

In [3]:
os.environ["OPENAI_API_KEY"] = "***"

In [4]:
TREND_LOC = "FR"

In [5]:
def get_trends(locale):
    url = f"https://trends.google.fr/trends/trendingsearches/daily/rss?geo={locale}"
    trend_response = requests.get(url=url)
    soup = BeautifulSoup(trend_response.text, "xml")

    items = soup.findAll("item")
    return [item.title.text for item in items]

In [6]:
def get_wiki(topic, n=5, loc="fr"):
    wikipedia.set_lang(loc)
    return topic+":"+",".join(wikipedia.search(topic, results = n))

In [7]:
trends = get_trends('FR')
contexts = [get_wiki(trend, n=5, loc="fr") for trend in trends]

In [8]:
template = """Generate your answer based only on the following context (delimited by <context> ... </context> tags):
<context>
{context}
</context>
Question: can you group these keywords in named categories: {trends}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "trends"],
)

model = ChatOpenAI()
chain = prompt | model

context = "\n".join(contexts)
trend_str = ",".join(trends)
llm_output = chain.invoke({"context": context, 'trends': trend_str})
print(llm_output.content)

Category 1: People
- Benoit Kounkoud
- Elon Musk
- Iskandar Safa
- Alain Delon
- Amelia Earhart
- Lorie
- Claude Lanzmann
- Xavier Niel
- Marine Le Pen
- Lamine Camara
- Emmanuel Macron
- Helena Noguerra
- Vitinha
- Arthur Cazaux

Category 2: Sports
- Benoit Kounkoud
- Maroc Afrique du Sud
- Mali-Burkina Faso
- Premier League
- Mali
- Lamine Camara
- Arthur Cazaux

Category 3: Technology
- Free
- Neuralink
- Elon Musk
- Xavier Niel

Category 4: Media and Entertainment
- Iskandar Safa Valeurs actuelles
- Alain Delon
- Avion Amelia Earhart
- Lorie
- Claude Lanzmann
- Helena Noguerra

Category 5: Political Figures
- Marine Le Pen
- Emmanuel Macron

Category 6: Miscellaneous
- RSA
